In [1]:
import backtrader as bt
import yfinance as yf
import pandas as pd

from utils.strategies import LongStraddleSimplified
from utils.tester import test_strategy, log_results, load_data

In [7]:
data_names = [
    # '^RUT', '^IXIC', '^VIX', 
    # '^DJI', '^GSPC', 
    # 'GOOG', 'MSFT', 'AMZN', 'TSLA', 
    # 'NFLX', 'NVDA', 'AMD'
    'BTC-USD', 'ETH-USD', 'BNB-USD', 'DOGE-USD', 'SOL-USD', 'TRX-USD', 'ADA-USD',
    'LINK-USD', 'BCH-USD', 'LEO-USD'
    ]

In [8]:
# Download SPX500 data from Yahoo Finance
yf_params = {'start': '2018-01-01', 'end': '2025-03-01',
             'interval': '1d', 'multi_level_index': False, 'auto_adjust': True}

feeds = load_data(data_names, yf_params)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

BTC-USD: from 2018-01-01 00:00:00 to 2025-02-28 00:00:00
ETH-USD: from 2018-01-01 00:00:00 to 2025-02-28 00:00:00
BNB-USD: from 2018-01-01 00:00:00 to 2025-02-28 00:00:00
DOGE-USD: from 2018-01-01 00:00:00 to 2025-02-28 00:00:00
SOL-USD: from 2020-04-10 00:00:00 to 2025-02-28 00:00:00
TRX-USD: from 2018-01-01 00:00:00 to 2025-02-28 00:00:00
ADA-USD: from 2018-01-01 00:00:00 to 2025-02-28 00:00:00
LINK-USD: from 2018-01-01 00:00:00 to 2025-02-28 00:00:00
BCH-USD: from 2018-01-01 00:00:00 to 2025-02-28 00:00:00
LEO-USD: from 2019-05-21 00:00:00 to 2025-02-28 00:00:00


In [9]:
strategy_params = dict(
                       atr_period=11,
                       atr_multiplier=1,
                       rrr=2.5,
)
cerebro_params = dict(percents=2)

results = dict()

for name, data_feed in feeds.items():
    results[name], cereb = test_strategy(LongStraddleSimplified, data_feed, strategy_params, cerebro_params)


2018-04-26 --- ENTRY SIGNAL ---
Potential Entry: 8867.32
Stop Loss Price: 8250.93
Take Profit Price: 10408.30
2018-04-27 BUY EXECUTED, Price: 8987.05, Cost: 2027.00, Comm: 2.03, Size: 0.22554728255171508
2018-05-12 SELL EXECUTED, Price: 8250.93, Cost: 2027.00, Comm: 1.86, Size: -0.22554728255171508
2018-05-12 Order Canceled/Margin/Rejected
2018-05-12 TRADE PROFIT, GROSS -166.03, NET -169.92
2018-07-17 --- ENTRY SIGNAL ---
Potential Entry: 6739.65
Stop Loss Price: 6384.19
Take Profit Price: 7628.31
2018-07-18 BUY EXECUTED, Price: 7370.78, Cost: 2183.57, Comm: 2.18, Size: 0.296247083599025
2018-07-23 SELL EXECUTED, Price: 7711.11, Cost: 2183.57, Comm: 2.28, Size: -0.296247083599025
2018-07-23 Order Canceled/Margin/Rejected
2018-07-23 TRADE PROFIT, GROSS 100.82, NET 96.35
2018-07-23 --- ENTRY SIGNAL ---
Potential Entry: 7414.71
Stop Loss Price: 7074.62
Take Profit Price: 8264.94
2018-07-24 BUY EXECUTED, Price: 8424.27, Cost: 2270.64, Comm: 2.27, Size: 0.26953565481619207
2018-07-25 SELL E

In [10]:
for name, res in results.items():
    log_results(name, res)



==================== BTC-USD ====================
Sharpe Ratio: -0.9003476211145053
Max Drawdown: 1.7716384806516041
Number of Trades: 134
Winning Trades: 67
Losing Trades: 67
Average Trade Return: 0.0021294996793970234
Total Returns: 0.02208273051942642
Final Balance: 102232.83587280958


==================== ETH-USD ====================
Sharpe Ratio: -0.36763515226728677
Max Drawdown: 1.5388389547409143
Number of Trades: 156
Winning Trades: 78
Losing Trades: 78
Average Trade Return: 0.004719800934417414
Total Returns: 0.04888076553395468
Final Balance: 105009.51357423978


==================== BNB-USD ====================
Sharpe Ratio: 0.07827310188270936
Max Drawdown: 1.7749625747144226
Number of Trades: 162
Winning Trades: 77
Losing Trades: 85
Average Trade Return: 0.00867984246610335
Total Returns: 0.08971623077462458
Final Balance: 109386.38347663083


==================== DOGE-USD ====================
Sharpe Ratio: 0.07280490527225192
Max Drawdown: 4.121202016974233
Number of 